# Swap Samples

This notebook demonstrates how to access and use the financial-contracts end point of Instrument Pricing Analytics (IPA) in the Refinitiv Data Platform.

##### Learn more

Please visit the [Refinitiv Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-ir-swap-contracts) to access the API documentation and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Refinitiv Data Platform Q&A Forum](https://community.developers.refinitiv.com/spaces/231/index.html). The Refinitiv Developer Community will be very pleased to help you. 

#### Import the library and connect to the Refinitv Data Platform

This sample works in both desktop and non-desktop environment. The appropriate session can be created based on the constant variable IPA_ENVIRONMENT, that is set by default to DESKTOP for execution in the Refinitiv codebook application. To run this sample in a non-desktop environment, you may set the variable to NON-DESKTOP.

In [4]:
import refinitiv.dataplatform as rdp

# Set IPA_ENVIRONMENT to NON-DESKTOP to use this example in a non desktop environment
# IPA_ENVIRONMENT = "NON-DESKTOP"
IPA_ENVIRONMENT = "DESKTOP"

if (IPA_ENVIRONMENT == "DESKTOP"):
    session = rdp.DesktopSession(app_key='DEFAULT_CODE_BOOK_APP_KEY')
else:
    %run ".\..\..\credentials-ss.ipynb"
    session = rdp.PlatformSession(APP_KEY, rdp.GrantPassword(RDP_LOGIN, RDP_PASSWORD))

session.open()
endpoint = rdp.Endpoint(session, "https://api.edp.thomsonreuters.com/data/quantitative-analytics/v1/financial-contracts")
print("session opened and end-point set")

session opened and end-point set


### Pricing a Vanilla IRS

This sample price a swap defined leg by leg for a complete control of its term and conditions. The 'fields' requested are:
- the gross price in percent (DirtyPricePercent), which is set to null for the solving process
- the fixedRatePercent of the fixed leg
- the spreadBp over the index of the float leg
- the PV01 (PV01Bp,PV01AmountInDealCcy)
- the names of the zero-coupon curves used (DiscountCurveName,ForwardCurveName)
- the error information (ErrorCode,ErrorMessage)

Solving Workflow:
- if neither fixedRatePercent nor spreadBp are defined in the legs, spreadBp is considered null and fixedRatePercent is solved so that the DirtyPricePercent of both legs match
- if the spreadBp value is set, fixedRatePercent is solved so that the DirtyPricePercent of both legs match
- if fixedRatePercent value is set, spreadBp is solved so that the DirtyPricePercent of both legs match


In [8]:
request_body = {
    
    "fields" : ["InstrumentTag","DirtyPricePercent","FixedRatePercent","SpreadBp","PV01Bp",\
                "PV01AmountInDealCcy","DiscountCurveName","ForwardCurveName","ErrorCode","ErrorMessage"],
    
    "universe" : [
        {
            "instrumentType":"Swap",
            "instrumentDefinition": {
                "instrumentTag":"IRS-EURIBOR-5Y",
                "startDate":"2020-01-28",
                "tenor":"5Y",
                "legs":[
                {
                    "direction":"Paid",
                    "interestType":"Fixed",
                    "notionalCcy":"EUR",
                    "interestPaymentFrequency":"Annual",
                    "interestCalculationMethod":"Dcb_30_360"
                },
                {
                    "direction":"Received",
                    "interestType":"Float",
                    "interestPaymentFrequency":"SemiAnnual",
                    "interestCalculationMethod":"Dcb_Actual_360",
                    "notionalCcy":"EUR",
                    "indexName":"EURIBOR",
                    "indexTenor":"6M",
                    "spreadBp":0.0
                }]
            }
        }],
    
    "pricingParameters": {
        "valuationDate": "2020-01-26T00:00:00Z"
    },
        
    "outputs" : ["Data","Headers"],

}

response = endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body)

print(response.data.raw)

{'headers': [{'type': 'String', 'name': 'InstrumentTag'}, {'type': 'Float', 'name': 'DirtyPricePercent'}, {'type': 'Float', 'name': 'FixedRatePercent'}, {'type': 'Float', 'name': 'SpreadBp'}, {'type': 'Float', 'name': 'PV01Bp'}, {'type': 'Float', 'name': 'PV01AmountInDealCcy'}, {'type': 'String', 'name': 'DiscountCurveName'}, {'type': 'String', 'name': 'ForwardCurveName'}, {'type': 'String', 'name': 'ErrorCode'}, {'type': 'String', 'name': 'ErrorMessage'}], 'data': [['IRS-EURIBOR-5Y', -1.12407582891065, -0.223000008008893, None, -0.0339193488435194, -3.39193488435194, 'EUR - Swap vs 6M Euribor', None, '', ''], ['IRS-EURIBOR-5Y', -1.12407582891065, None, 0.0, -4.56894321306987, -456.894321306987, 'EUR - Swap vs 6M Euribor', 'EUR - Swap vs 6M Euribor', '', '']]}


In [9]:
import pandas as pd

headers_name = [h['name'] for h in response.data.raw['headers']]
df = pd.DataFrame(data=response.data.raw['data'], columns=headers_name)
df

,InstrumentTag,DirtyPricePercent,FixedRatePercent,SpreadBp,PV01Bp,PV01AmountInDealCcy,DiscountCurveName,ForwardCurveName,ErrorCode,ErrorMessage
0,IRS-EURIBOR-5Y,-1.124076,-0.223,NaN,-0.033919,-3.391935,EUR - Swap vs 6M Euribor,None,,
1,IRS-EURIBOR-5Y,-1.124076,NaN,0.0,-4.568943,-456.894321,EUR - Swap vs 6M Euribor,EUR - Swap vs 6M Euribor,,


2021-01-11 18:06:58,769 - Session session.platform - Thread 21932 | MainThread
[Error 400 - invalid_grant] empty error description
2021-01-11 18:06:58,769 - Session session.platform - Thread 21932 | MainThread
[Error 400 - invalid_grant] empty error description
